# CSE 6240/CX4803 Web Search and Text Mining
## Homework 2: Link analysis and Information Cascades

This homework asks you to perform network analysis on a real world social network.

In [1]:
import networkx as nx
import math
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.stats import pearsonr, spearmanr
%matplotlib inline

Please run the following cell substituting your student and user names.

In [2]:
def author_honor_code (student_name='Jiarui Xu', user_name='jxu605'):
  print (f'I, {student_name} ({user_name}), state that I performed the tasks in this assignment following the Georgia Tech honor code(https://osi.gatech.edu/content/honor-code).')

# print the honor code before submission (substitute your name and username)
author_honor_code ()

I, Jiarui Xu (jxu605), state that I performed the tasks in this assignment following the Georgia Tech honor code(https://osi.gatech.edu/content/honor-code).


### Data Loading: GitHub network

Throughout this assignment, we will use a large social network from [GitHub](github.com) again . The following cell will download the network in your current directory. If it does not work, please download the zip file from [here](http://snap.stanford.edu/data/git_web_ml.zip) and unzip.

The GitHub network can be described as follows:

- A node in the network represents a developer account that is active.
- Two nodes are linked if they mutually follow each other.

There is other information related to the nodes in the data you just downloaded, but for the purpose of this assignment we will ignore it.

In [3]:
![ -d "git_web_ml" ] && rm -rf git_web_ml.zip && rm -rf git_web_ml
!wget http://snap.stanford.edu/data/git_web_ml.zip
!unzip git_web_ml.zip

--2022-02-08 21:50:25--  http://snap.stanford.edu/data/git_web_ml.zip
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2396031 (2.3M) [application/zip]
Saving to: ‘git_web_ml.zip’

git_web_ml.zip      100%[===================>]   2.28M  2.13MB/s    in 1.1s    

2022-02-08 21:50:26 (2.13 MB/s) - ‘git_web_ml.zip’ saved [2396031/2396031]

Archive:  git_web_ml.zip
   creating: git_web_ml/
  inflating: git_web_ml/musae_git_edges.csv  
  inflating: git_web_ml/musae_git_features.json  
  inflating: git_web_ml/musae_git_target.csv  
  inflating: git_web_ml/citing.txt   
  inflating: git_web_ml/README.txt   


In [4]:
def load_net (filename):
  """ Load the network from the file.
  Arguments
  ---------
  filename (str): The name of the file which 
  contains the edges.

  Returns
  -------
  networkx.Graph

  Steps:
  1. Split each line by comma.
  2. Add edges to the network.
  """
  net = nx.Graph()
  with open (filename) as fin:
    for i, line in enumerate (fin):
      if i > 0:
        parts = line.strip().split (',') 
        net.add_edge (parts[0], parts[1])
  return net

In [5]:
net = load_net ('git_web_ml/musae_git_edges.csv')

### Section 1: Centrality measures [1 point]

A common question in network analysis is to find the central nodes in the network. These nodes are likely to be more important and more influential. In this section, we'll first calculate the degree, betweenness, and eigenvector centrality for the GitHub network by calling `networkx` functions. 

The betweenness centrality calculation can be extremely slow. To speed it up, we use an approximation which is parameterized by the variable $k$ in `networkx`. Please use $k=100$  

In [10]:
## Add code to calculate the different centrality measures [0.4 points] ##
degree_centrality=nx.degree_centrality(net)
betweenness_centrality=nx.betweenness_centrality(net,k=100)
eigenvector_centrality=nx.eigenvector_centrality(net)
###########################################################################

In [6]:
def get_most_central (centrality_scores, top_k=10):
  """ Given the centrality scores, give the top_k
      nodes with the highest centrality as a list of tuples.
      Each tuple should be the id of the node and the centrality score
  """
  return list(sorted (centrality_scores.items(), key=lambda x:x[1], reverse=True))[0:top_k]

In [11]:
print (get_most_central (degree_centrality, top_k=5))
print (get_most_central (betweenness_centrality, top_k=5))
print (get_most_central (eigenvector_centrality, top_k=5))

[('31890', 0.25088198625958247), ('27803', 0.18793601952306427), ('35773', 0.08817210005570439), ('19222', 0.07846361972466113), ('13638', 0.06546592747818245)]
[('31890', 0.2653892482871484), ('27803', 0.24709706937780054), ('19222', 0.04756106345132798), ('10001', 0.047068429270954844), ('35773', 0.04268982109070867)]
[('31890', 0.35594895516626784), ('27803', 0.3016546270787933), ('35773', 0.16167672528839555), ('19222', 0.14884182734999724), ('13638', 0.11946064577011832)]


Now calculate the correlation between these centrality metrics using `pearsonr` function from `scipy.stats`

Note: Make sure that the input to the functions are the centrality values for the same sequence of nodes.

In [14]:
## Add code to calculate the pearson correlation between the different centrality measures [0.2 points] ##
def GetItem(input):
  result=np.zeros(len(input))
  for i in range(len(input)):
    result[i]=input[i][1]
  return result
a=get_most_central (degree_centrality)
b=get_most_central (betweenness_centrality)
c=get_most_central (eigenvector_centrality)
a=GetItem(a)
b=GetItem(b)
c=GetItem(c)
print('degree_centrality and betweenness_centrality:')
print(pearsonr(a,b))
print('degree_centrality and eigenvector_centrality:')
print(pearsonr(a,c))
print('betweenness_centrality and eigenvector_centrality:')
print(pearsonr(b,c))
##########################################################################################################

degree_centrality and betweenness_centrality:
(0.9776333433236849, 1.0658052498769655e-06)
degree_centrality and eigenvector_centrality:
(0.9925197219361774, 1.3575170691232037e-08)
betweenness_centrality and eigenvector_centrality:
(0.977343137231363, 1.1218122200208991e-06)


Explain in a paragraph what these centrality measures mean. From the top central nodes and the results about the correlation between the centrality measures, tell us why the results are expected or unexpected? [0.4 points]

Answer: 

Degree centrality: Identify the node that has the most followers. The more followers the node has, the more important it is.

Betweenness centrality: Identify the node which will see the most information, if information flows via shortest paths.  

Eigenvector centrality: The basic idea of eigenvector centrality is that the centrality of a node is a function of the centrality of adjacent nodes.  In other words, the more important the node connect with, the more important it is.

The result of correlation is expected. Their correlation are very close, because the nodes with most followers will be more likely to have more connection and will be more likely to be close to other nodes.

### Section 2: Simplified PageRank and HITS  [4 points]

In this section, we'll then move to link analysis and implement a simplified versions of the PageRank and HITS algorithm to find the more important nodes in the network. Finally, we'll compare our PageRank scores with `networkx` Pagerank scores and scores from the HITS algorithm.



#### 2.1 PageRank algorithm [1.5 points]

Now, we'll switch gears to a different, more principled way of assigning importance scores to each node: Pagerank. The idea behind Pagerank is that a node is more important if other important nodes link to it. It's also the basic algorithm upon which Google ranked webpages for their search results. We'll implement a simplified iterative version of Pagerank.

To get started, we will first convert the network into a directed network
(This is not strictly necessary, but PageRank and HITS work best on a directed network). We will simply convert the edges to directed edges by making them point to nodes that have a higher id.

In [24]:
def convert_to_directed (net):
  G = nx.DiGraph ()
  for source, target in net.edges ():
    if int (source) > int (target):
      G.add_edge (source, target)
    else:
      G.add_edge (target, source)
  
  return G

In [73]:
di_net = convert_to_directed (net)
A = nx.linalg.graphmatrix.adjacency_matrix (di_net)
nodeid2rownum = {node: i for i, node in enumerate (di_net.nodes())}
rownum2nodeid = {i: node for i, node in enumerate (di_net.nodes())}

You'll implement PageRank as the following iterative algorithm.

$$
R_i^{(0)} = \frac{1}{N} \\
R_i^{(t+1)} = \frac{1-p}{N} + p \sum\limits_{j \in M(i)} \frac{R_j^{(t)}}{k_j^{\text{out}}}, 
$$

where $R_{i}$ is the PageRank of the node $i$; $N$ is the total number of nodes in the network; $p$ is the transportation probability; $M(i)$ is the set of nodes that have an edge incident on node $i$; $k_j^{\text{out}}$ is the out-degree of node $j$.

In [26]:
def init_pagerank_scores (index):
  """ Initialize the pageranks for each node. 
  Should return a dictionary of nodes as key and value as the pagerank.

  Arguments
  ---------
  index (tuple): maps the node id to row number in the adjacency matrix and 
                 vice versa

  Returns
  -------
  scores (dict): initial pagerank scores per node. 
                 Node id is the key and the score is the value.
  """
  idx, iidx = index
  scores = None
  ## Add code below to initialize the pagerank [0.2 points] ##
  scores={}
  for i in range(len(list(idx.values()))):
    scores[iidx[i]]=1/len(list(idx.values()))
  ############################################################  
  return scores

def pagerank_contribution (A, index, scores, from_node):
  """ Calculate the contribution to pagerank score of any node "from_node".

  Arguments
  ---------
  A (scipy.sparse.matrix): The adjacency matrix
  index (tuple): The mapping from node id to row number in adjacency matrix 
                 (and vice versa)
  scores (dict): The pagerank scores from the previous iteration
  from_node (string): The id of the node whose contribution is to be calculated.

  Returns
  ------
  contribution (float): The contribution of "from_node"
  """
  idx, iidx = index
  contribution = None
  ## Add code below to calculate the contribution from one neigboring node [0.4 points] ##
  k=np.sum(A[idx[from_node]])
  contribution=scores[from_node]/k
  ########################################################################################
  return contribution

def pagerank_for_node (A, index, scores, for_node, p):
  """ Calculate the pagerank for node "for_node" in this iteration.

  Arguments
  ---------
  A (scipy.sparse.matrix): The adjacency matrix
  index (tuple): The index to map from node ids to row numbers and vice versa
  scores (dict): The pagerank scores from the previous iteration
  for_node (string): The id of the node for which pagerank is to be calculated.
  p (float): The transportation probability in the random walk

  Returns
  -------
  score (float): The pagerank score for this node

  """
  idx, iidx = index
  score = None
  ## Add code to calculate the pagerank for the given node in this iteration [0.5 points] ##
  N=len(idx)
  sum=0
  A2=[node for node in A[:,idx[for_node]].nonzero()[0]]
  for i in range(len(A2)):
    sum+=pagerank_contribution(A, index, scores, iidx[A2[i]])
  score=(1-p)/N+p*sum
  ##########################################################################################
  return score

def error_within_tolerance (old_scores, new_scores, tol):
  """
  Check if new pagerank scores are within tolerance with respect to old pagerank scores

  Arguments
  ---------
  old_scores (dict): The pagerank scores from previous iteration
  new_scores (dict): The pagerank scores from ongoing iteration
  tol (float): The tolerance limit

  Returns
  -------
  bool: True if error is within tolerance, False if error is outside tolerance
  """
  old_vec = np.array([old_scores[node] for node in old_scores])
  new_vec = np.array([new_scores[node] for node in old_scores])
  return (np.linalg.norm(old_vec - new_vec) < tol)

def simplified_pagerank (G, A, index, p=0.85, num_iters=50, tol=1e-3):
  """
  Calculate the pagerank scores for a given graph.

  Arguments
  ---------
  G (networkx.DiGraph): The directed network for which pagerank is to be calculated.
  A (scipy.sparse.matrix): The adjacency matrix for the given directed network.
  index (tuple): The index maps the node id in the graph to row 
                 number in the adjacency matrix and vice versa.
  p(float): The random walk restart probability. It's typically set to 0.85 for convergence.
  num_iters (int): The number of iterations to perform
  tol (float): The tolerance value to keep track; algorithm should exit if 
               either the number of iterations are exhausted or difference between
               the pagerank scores across two consecutive iterations is within tolerance.
  Returns
  -------
  scores (dict): The pagerank scores as key-value pairs in a dictionary.
  """

  scores = init_pagerank_scores (index)
  ## Add code to piece it all together [0.4 points] ##
  idx, iidx = index
  iter_times=0
  N=len(idx)
  new_scores={}
  while iter_times<=num_iters:
    iter_times+=1
    for node in G.nodes():
      new_scores[node]=pagerank_for_node (A, index, scores, node, p)
    if not error_within_tolerance (scores, new_scores, tol):
      scores=new_scores
    else:
      break
  ####################################################
  return scores

In [27]:
pagerank_scores = simplified_pagerank (di_net, A, (nodeid2rownum, rownum2nodeid), p=0.85, num_iters=3, tol=1e-5)

In [28]:
print (get_most_central (pagerank_scores, top_k=5))

[('73', 0.009710435037992574), ('2078', 0.00495014465590489), ('61', 0.004571853725305612), ('42', 0.00445365069871951), ('3712', 0.0030292102005308533)]


#### 2.2 HITS algorithm [1.5 points]

You'll implement the HITS algorithm as the following iterative algorithm.

$$
A_{i}^{(0)} = 1, \\ 
H_{i}^{(0)} = 1, \\
\\
A_{i}^{(t+1)} = \sum\limits_{j \in M(i)} H_{j}^{(t)}, \\
H_{i}^{(t+1)} = \sum\limits_{j \in Q(i)} A_{j}^{(t)},
$$

where $A_{i}$ and $H_{i}$ are the authority and hub score of the node $i$; $M(i)$ is the set of nodes that have an edge incident on node $i$; $Q(i)$ is the set of nodes that have an edge from node $i$.


After each iteration, it is important to normalize the authority and hub scores over all the nodes to make the algorithm converge.

Thus, let $N_{a}^{(t+1)}$ and $N_{h}^{(t+1)}$ be the normalization for the authority and hub scores respectively. These are calculated as:

$$
N_{a}^{(t+1)} = \sqrt{\sum_i (A_i^{(t+1)})^{2}}, \\
N_{h}^{(t+1)} = \sqrt{\sum_i (H_i^{(t+1)})^{2}}, \\
$$

$$
A_{i}^{(t+1)} = \frac{A_{i}^{(t+1)}}{N_{a}^{(t+1)}} \forall i \in \mathcal{V}, \\
H_{i}^{(t+1)} = \frac{H_{i}^{(t+1)}}{N_{h}^{(t+1)}} \forall i \in \mathcal{V}, 
$$

where $\mathcal{V}$ is the set of nodes in the graph.

In [80]:
def init_hub_authority_scores (index):
  """ Initialize the hub and authority scores per node

  Arguments
  ---------
  index (tuple): The mapping from node ids in the graph to adjacency 
                 matrix row numbers  (and vice versa)
  
  Returns
  -------
  hub_scores (dict), auth_scores (dict): The hub and authority scores as
                                         separate dictionaries.
  """
  idx, iidx = index
  hub_scores, auth_scores = None, None
  ## Add code to initialize hub and authority scores [0.2 points] ##
  hub_scores={}
  auth_scores={}
  for i in range(len(list(idx.values()))):
    hub_scores[iidx[i]]=1
    auth_scores[iidx[i]]=1
  ##################################################################
  return hub_scores, auth_scores

def normalize (scores):
  """ Normalize the scores
  Arguments
  ---------
  scores (dict): Score per node as key-value pairs

  Returns
  -------
  normalized_scores (dict): Score per node as key-value pairs but normalized
  """
  normalized_scores = None
  ## Add code to normalize the scores [0.2 points] ##
  value=np.array(list(scores.values()))
  N=np.sqrt(np.sum(value*value))
  normalized_scores={}
  for item in scores.keys():
    normalized_scores[item]=scores[item]/N
  ##################################################
  return normalized_scores

def auth_score_for_node (A, index, hub_scores, for_node):
  """ Calculate the authority score for "for_node" in this iteration.
  Arguments
  ---------
  A (scipy.sparse.matrix): adjacency matrix
  index (tuple): mapping between node ids and adjacency matrix row numbers
  hub_scores (dict): The hub scores from the previous iteration
  for_node (string): The id of the node for which the hub score is to be calculated

  Returns
  -------
  score (float): The authority score for "for_node".
  """
  idx, iidx = index
  score = None
  ## Add code to calculate the authority score for the given node in this iteration [0.3 points] ##
  score=0
  A2=[node for node in A[:,idx[for_node]].nonzero()[0]]
  for i in range(len(A2)):
    score=score+hub_scores[iidx[A2[i]]]
  #################################################################################################
  return score

def hub_score_for_node (A, index, auth_scores, for_node):
  """ Calculate the hub score for "for_node" in this iteration.
  Arguments
  ---------
  A (scipy.sparse.matrix): adjacency matrix
  index (tuple): mapping between node ids and adjacency matrix row numbers
  auth_scores (dict): The authority scores from the previous iteration
  for_node (string): The id of the node for which the hub score is to be calculated

  Returns
  -------
  score (float): The hub score for "for_node".
  """
  idx, iidx = index
  score = None
  ## Add code to calculate the hub score for the given node in this iteration [0.3 points] ##
  score=0
  A2=[node for node in A[idx[for_node],:].nonzero()[1]]
  for i in range(len(A2)):
    score=score+auth_scores[iidx[A2[i]]]
  #################################################################################################
  return score

def simplified_hits (G, A, index, num_iters=50, tol=1e-3):
  """ Implement the simplified HITS algorithm

  Arguments
  ---------
  G (networkx.DiGraph): The directed network for which hubs and authority scores are to be computed
  A (scipy.sparse.matrix): The adjacency matrix.
  index (tuple): The mapping from node id (in the directed network) to row number (in the adjacency matrix)
                 and vice versa
  num_iters (int): The max number of iterations.
  tol (float): The tolerance limit. Exit the algorithm if num_iters is reached or scores from 
               current iteration differ from the previous iteration only upto tolerance specified.
  Returns
  -------
  hub_scores (dict), auth_scores(dict): The hub and authority scores as key value pairs in separate dictionaries. 
  """
  hub_scores, auth_scores = init_hub_authority_scores (index)
  auth_scores = normalize (auth_scores)
  hub_scores = normalize (hub_scores)

  ## Add code to piece it all together [0.5 points] ##
  idx, iidx = index
  iter_times=0
  new_auth_scores={}
  new_hub_scores={}
  while iter_times<=num_iters:
    iter_times+=1
    for for_node in hub_scores.keys():
      new_auth_scores[for_node]=auth_score_for_node(A, index, hub_scores, for_node)
      new_hub_scores[for_node]=hub_score_for_node(A, index, auth_scores, for_node)
    if error_within_tolerance (auth_scores, new_auth_scores, tol) and error_within_tolerance (hub_scores, new_hub_scores, tol):
      auth_scores = normalize (new_auth_scores)
      hub_scores = normalize (new_hub_scores)
      break
    else:
      auth_scores = normalize (new_auth_scores)
      hub_scores = normalize (new_hub_scores)
  ####################################################
  return hub_scores, auth_scores

In [81]:
hub_scores, auth_scores = simplified_hits (di_net, A, (nodeid2rownum, rownum2nodeid), num_iters=3, tol=1e-5)

In [82]:
print (get_most_central (hub_scores, top_k=5))
print (get_most_central (auth_scores, top_k=5))

[('31890', 0.49669068618516826), ('27803', 0.4234942955178548), ('35773', 0.23343583281447222), ('19222', 0.14590601865247244), ('36652', 0.13897126929996248)]
[('27803', 0.07188189108002344), ('19222', 0.07178699177674049), ('13638', 0.0697482935025386), ('7027', 0.05528717813708258), ('10001', 0.052089770229031974)]


#### 2.3 Comparison with `networkx` algorithms [1 point]

**Pagerank**

1. Calculate the pagerank scores using the algorithm in `networkx` [0.1 points]


In [29]:
## Add code below to get pageranks scores from networkx [0.1 points] ##
print(nx.pagerank(di_net,alpha=0.85))
#######################################################################

{'23977': 1.4284167481135854e-05, '0': 1.006841480088353e-05, '27803': 0.0019122111192680636, '69': 7.107973140336408e-05, '1966': 3.864795526569081e-05, '2939': 7.792088235692472e-05, '3147': 1.410353292358408e-05, '4422': 3.91182125158894e-05, '5631': 4.6402670458931274e-05, '5895': 6.593420952916058e-05, '25679': 1.0820777234478137e-05, '10111': 2.8804498680451937e-05, '8973': 1.976949233458035e-05, '9212': 3.480538846318635e-05, '10081': 9.983302422251197e-05, '11305': 4.661152193906165e-05, '12114': 5.063055002125467e-05, '13060': 1.662696056141041e-05, '14480': 1.2002987528987469e-05, '15313': 0.00014502163619533256, '16972': 3.372930970065183e-05, '18520': 2.094757266415559e-05, '19222': 0.0020539081719098914, '19375': 1.6945848089445847e-05, '30863': 1.1889505346910467e-05, '29826': 9.732405110498547e-06, '31890': 0.0019214802024749983, '17127': 4.0818336633403195e-05, '29188': 1.925728661211712e-05, '35828': 1.8975656097781455e-05, '25285': 2.7303357775596156e-05, '33206': 1.3

2. Calculate the pearson correlation between your own and `networkx` implementation. Report the result [0.1 points]

In [30]:
## Add code below to calculate the correlation between your implementation and networkx implementation [0.1 points] ##
a=np.array(list(nx.pagerank(di_net,alpha=0.85).values()))
b=np.array(list(pagerank_scores.values()))
print(pearsonr(a,b))
######################################################################################################################

(0.9839618970882028, 0.0)


3. Are the scores from your implementation strongly correlated with `networkx` scores? If yes, give 3 reasons why you were able to have strong correlation . If not, give 3 technical reasons why your results are different from `networkx` [0.2 points]

Answer:

Yes, they are strong correlated. Reasons: 1.The two use the same algorithm: pagerank. 2.The two use the same data to calculate. 3.The two use the same alpha(convergent boundary condition) which is 0.85.

**HITS**

4. Calculate the hubs and authority scores from `networkx` [0.1 points]


In [83]:
## Add code below to get hub and authority scores from networkx [0.1 points] ##
nx_hub_scores, nx_authority_scores = nx.hits(di_net, max_iter = 50, tol=1e-3, normalized = True) 
print(nx_hub_scores)
print(nx_authority_scores)
###############################################################################

{'23977': 5.1881889227755374e-05, '0': 0.0, '27803': 0.013186702028769813, '69': 0.0, '1966': 0.0, '2939': 2.1590119771879404e-05, '3147': 4.797841212928823e-06, '4422': 8.02287474550495e-06, '5631': 1.2734323241980788e-05, '5895': 3.7804905289630036e-05, '25679': 3.5603159646485686e-05, '10111': 1.4860045718888594e-05, '8973': 0.0, '9212': 1.4162848047324978e-05, '10081': 5.94556381571703e-05, '11305': 0.00010009576231260804, '12114': 6.956768442405567e-05, '13060': 1.4527667947273736e-05, '14480': 4.986458580576064e-07, '15313': 0.0003216555895109435, '16972': 8.220748804470173e-05, '18520': 1.619982196787644e-05, '19222': 0.003933534401514369, '19375': 6.953412474924613e-05, '30863': 0.00020801285914765283, '29826': 1.5209376788637555e-05, '31890': 0.02157237760050531, '17127': 6.659793382704045e-05, '29188': 0.00022183080728516925, '35828': 0.0010244061386831301, '25285': 9.589173939470237e-05, '33206': 0.0005446278450749425, '25477': 0.0016312762296647142, '34526': 6.1389612414949

5. Calculate the pearson correlation between your own and networkx implementation to get the hubs and authority scores. Report the result [0.1 points]

In [84]:
## Add code to calculate the pearson correlation [0.1 points] ##
h1=np.array(list(nx_hub_scores.values()))
h2=np.array(list(hub_scores.values()))
print(pearsonr(h1,h2))
a1=np.array(list(nx_authority_scores.values()))
a2=np.array(list(auth_scores.values()))
print(pearsonr(a1,a2))
################################################################

(0.9495890567871348, 0.0)
(0.999647166745016, 0.0)


6. Are the scores from your implementation strongly correlated with `networkx` scores? If yes, give three reasons why that's the case? If no, give three reasons why the scores are not strongly related? [0.2 points]

Answer

Yes, they are strongly correlated. Reasons: 1.The two use the same algorithm: HITS. 2. The two use the same data. 3.Both two do normalization and they iterate in the same way.

7. Give two cases each of when would you prefer PageRank over HITS and vice versa? [0.2 points]

Answer

Prefer PageRank:
1.I want to be more efficient. Pagerank is static, so it is much more efficient than HITS.
2.I want to have a stable system. Because pagerank is more stable than HITS.
3.I don't want to see tricks. HITS is easier to have tricks. HITS easily manipulated by cheaters from the mechanism, such as the cheater can create a web page, the page content increase many point to the high quality of the pages or the famous website url, this is a good Hub page, cheaters will again after this web page link to cheat, cheat so can promote Authority score of a web page.

Prefer HITS:
1.I want to query something. HITS is query-dependent and Pagerank is not. HITS is a collection of web pages that are constructed based on keywords entered into a query, so its search results are related to the content of the query.
2.I want to search within a few webpage instead of all the pages. Because PageRank involves all web pages, HITS algorithm will search in fewer pages.

### Section 3: Personalized PageRank [2 points]

Personalized PageRank algorithm is an extension of the PageRank algorithm where the teleportation set of the random walker is restricted to a set $S$ of nodes. Given the teleportation set $S$ of nodes, we can calculate the stationary distribution of a random walker visiting all nodes in the graph. This distribution over all nodes with respect to $S$ is called the Personalized PageRank Distribution. This is represented as vector $PPRD(* | S )$.

Assume two disjoint teleportation sets $S_1$ and $S_2$, such that $S_1 ∩ S_2 = \phi$. The Personalized PageRank Distribution of the set $S_1 \cup S_2$ can be calculated as $PPRD(* | S_1 \cup S_2 ) = PPRD(* | S_1 ) + PPRD(* | S_2 )$. This is because the distributions are stationary and the contributions of nodes in the two sets is complementary. 

Suppose you have already calculated the Personalized PageRank Distribution vectors for various set teleportation sets $S$:

- $PPRD( * | \{1, 2, 3\})$ for $S = \{1, 2, 3\}$
- $PPRD( * | \{3, 4, 5\})$ for $S = \{3, 4, 5\}$
- $PPRD( * | \{1, 4, 5\})$ for $S = \{1, 4, 5\}$
- $PPRD( * | \{1\})$ for $S = \{1\}$

During the random walks, we assume that the weight for each node in the teleport set
is uniform and there is a fixed teleport parameter $\beta$.

1. Without having access to the graph, can $PPRD ( * | {2} )$ be calculated? If so, how? If not, why not? [1 point]

Answer 
Yes, as 1,2,3,4,5 all individual nodes:

$PPRD( * | \{2\})$

=$PPRD( * | \{1, 2,3\})$ - $PPRD( * | \{1,3\})$

=$PPRD( * | \{1, 2,3\})$ - $PPRD( * | \{1\})$ - $PPRD( * | \{3\})$

=$PPRD( * | \{1, 2,3\})$ - $PPRD( * | \{1\})$ - ($PPRD( * | \{3, 4,5\})$ - $PPRD( * | \{4,5\})$) 

= $PPRD( * | \{1, 2,3\})$ - $PPRD( * | \{1\})$ - ($PPRD( * | \{3, 4,5\})$ - ($PPRD( * | \{1,4,5\})$ - $PPRD( * | \{1\})$))

2. Without having access to the graph, can $PPRD ( * | {5} )$ be
calculated? If so, how? If not, why not? [1 point]

Answer 
No, beacuse $PPRD( * | \{4\})$ is unknown; node 4 and 5 always be together, so we cannot calculate $PPRD( * | \{5\})$.

### Section 4: Diffusion on a network [3 points]


Now we'll learn about diffusion on a network. One scenario when diffusion is seen is when nodes in the network make decisions based on their local neighborhood.

To motivate this part, consider the undirected GitHub network. 
Imagine that these developers are electing the next GitHub CEO. 
There are two candidates Anna Annoying and Bob Bigface, but let's call them 'A' and 'B' respectively. 

Initially, some developers have a preference for 'A' or 'B' but there are also some who are undecided (let's call that preference as 'U').

Then for a fixed number of rounds, the preferences are updated as follows.
- If the node preference is 'A' or 'B' from the previous round, then that preference remains the same 
- If the node preference is 'U' from the the previous round, then 
  - the preference is updated to 'A' if majority neighbors have preference 'A'
  - the preference is updated to 'B' if majority neighbors have prefefence 'B'
  - the preference remains 'U' if majority neighbors have preference 'U' 

After a fixed number of rounds, 'A' wins if the number of nodes whose preference is 'A' exceeds that of 'B', 'B' wins if the number of nodes whose preference is 'B' exceeds that of 'A', otherwise it is a tie. 


1. Let's first write some utulity functions to write who will be winner of the election. [0.4 points]

Please read the docstring for each function.

In [7]:
def get_neighbors(G, node):
    """
    Function to get all neighbours of a node in a given graph.

    Arguments
    ---------
    G (networkx.Graph): The given graph
    node (string) : The node id of the graph

    Returns
    -------
    neighbours (list): list of neighbouring nodes
    """
    neighbours = [neighbor for neighbor in G.neighbors (node)]
    return neighbours

def get_vote_count(nodeset, pref, candidate):
    """ Function to get the vote counts for a particular candidate (letter) 
    given a nodeset and their preferences

    Arguments
    ---------
    nodeset (iterable): set of nodes in the graph
    pref (dict) : a mapping of the preference for each node
    candidate (string): Candidate letter

    Returns
    -------

    vote_count : Vote count for the given candidate

    """
    vote_count = sum([pref[n] == candidate for n in nodeset])
    return vote_count

def winner(pref):
    """ Function to get the winner of election process.
    (Please use the appropriate utitlity function(s) as required.)

    Arguments
    ---------
    
    pref (dict): Dictionary object mapping node ids to the voting preferences

    Returns
    -------
    winner (string): Winning candidate character (char)
    margin (int): Margin of victory or loss for candidate 'A'

    Note: Please note that for margin calculation we're NOT taking the absolute value.
    """
    winner = None
    margin = None
    ## Add your code here.[0.4 Points] ##
    A=0
    B=0
    for item in pref.values():
      if item=='A':
        A+=1
      elif item=='B':
        B+=1
      else:
        pass
    if A>B:
      winner='A'
      margin=A-B
    elif B>A:
      winner='B'
      margin=A-B
    else:
      winner=None
      margin=0

    #####################################
    return winner,margin

2. Now write functions to initialize the voting preferences, iterate the voting for a fixed number of decision steps, and a function to simulate the entire election [1.8 points].

Again, please look at the docstrings to understand the requirements of each of the functions.

In [8]:
def initial_voting_state(G, undecided_proportion=0.5):
    """
    Function to initialize the voting preferences.

    The allotment of initial voting preferences is done as follows.

    The node ids (treated as an integer) are mapped to voting preferences
    such that if undecided proportion is x then the first (1-x)/2 nodes go for 'A',
    the next x go for 'U' and the last (1-x)/2 go for 'B'. 

    Arguments
    ---------
    Graph (networkx.Graph): an undirected graph
    undecided_proportion (float): A value between 0 and 1.
                                  0 corresponds to 0%, 1 corresponds to 100%
                                  This proportion of users are initially undecided

    Returns
    -------
    voter_prefs: Dictionary mapping node IDs to initial voter preference
            ('A', 'B', or 'U')
    Note: 'U' denotes undecided voting preference.

    Example: Some random key-value pairs of the dict are
             {0 : 'A', 24 : 'B', 118 : 'U'}.
    """
    voter_prefs = {}

    ### Add your code here.[0.4 Points] ##
    nodes=[node for node in G.nodes()]
    N=len(nodes)
    A=int(N*(1-undecided_proportion)/2)
    B=int(N*(1-undecided_proportion)/2)
    U=N-A-B
    for i in range(N):
      if i <A:
        voter_prefs[nodes[i]]='A'
      elif i<A+U:
        voter_prefs[nodes[i]]='U'
      else:
        voter_prefs[nodes[i]]='B'
    ######################################
    return voter_prefs

def iterate_voting(G, init_pref, decision_steps=10):
    """
    Function to perform the 10 step decision process.

    Arguments
    ---------
    G (networkx.Graph): An undirected graph
    init_pref (dict): Dictionary object containing the initial voting
                      preferences (before any iteration of the decision
                      process)
    decision_steps (int): The number of iterations to make

    Returns
    -------
    curr_pref (dict): Dictionary containing the voting preferences 
                      (mapping node IDs to 'A','B' or 'U') after the decision process.

    """
    curr_pref = deepcopy(init_pref)

    ### Add your code here.[1 Point] ##
    iter_times=0
    nodes=[node for node in G.nodes()]
    N=len(nodes)
    update={}
    while iter_times<decision_steps:
      iter_times+=1
      for i in range(N):
        if curr_pref[nodes[i]]=='U':
          A=get_vote_count(get_neighbors(G, nodes[i]), curr_pref, 'A')
          B=get_vote_count(get_neighbors(G, nodes[i]), curr_pref, 'B')
          U=get_vote_count(get_neighbors(G, nodes[i]), curr_pref, 'U')
          if not (A<=U and B<=U):
            if A>B:
              update[nodes[i]]='A'
            elif B>A:
              update[nodes[i]]='B'
      for key,value in update.items():
        curr_pref[key]=value
    ######################################
    return curr_pref

def sim_election(G, decision_steps=10, undecided_proportion=0.5):
    """
    Function to simulate the election process, takes the Graph as input and
    gives the final voting preferences (dictionary) as output.

    Arguments
    --------- 
    G (networkx.Graph) : the undirected network
    decision_steps (int): The number of iterations (default=10)
    undecided_proportion (float): The number of undecided voters initially (default=0.5)
    
    
    Returns
    -------
    
    pref (dict): Dictionary containing the voting preferences (mapping node IDs to
                        'A','B' or 'U') after the decision process.
    """
    pref = None
    ### Add your code here.[0.4 Points]
    pref=initial_voting_state(G, undecided_proportion)
    pref=iterate_voting(G, pref, decision_steps)
    ######################
    return pref

In [9]:
for i in range(1,10):
  pref = sim_election (net, undecided_proportion=i/10)
  print(i/10)
  print (winner (pref))

0.1
('A', 3579)
0.2
('A', 7172)
0.3
('A', 10871)
0.4
('A', 14570)
0.5
('A', 18336)
0.6
('A', 22063)
0.7
('A', 25815)
0.8
('A', 29467)
0.9
('A', 29613)


Finally, answer the folllowing questions. 

3. Vary the number of undecided voter proportion initially from $0.1$ to $0.9$ in increments of $0.1$. In how many cases does 'A' win? Similarly, in how many cases does 'B' win? [0.2 points]

Answer

All the cases A wins. No cases B wins.

4. At what undecided proportion you get the biggest margin of win? [0.2 points]

Answer

At undecided proportion equals to 0.9, I got the biggest margin of win.

5. Does the margin of the win increase or decrease as the number of undecided voters increase? Why does it increase or decrease? [0.4 points]

Answer

The undecided voters proportion increase from 0.1 to 0.9, the margin increase.  I think the winner will gets more support from people becase the winners' supporter will influence the undecided voters and the more undecided voters, the more people the supporters will influence. The first few node has strong influence on other nodes and they all support A, so A's supporters will spread widely. The more undecided voters at first, the more voters will become A's supporters in the end.